In [ ]:
%load_ext google.cloud.bigquery
%load_ext tensorboard
import os
import pandas as pd
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="gcloud-credentials.json"

# "Will it snow tomorrow?" - The time traveler asked
The following dataset contains climate information form over 9000 stations accross the world. The overall goal of these subtasks will be to predict whether it will snow tomorrow 12 years ago. So if today is 2021.02.15 then the weather we want to forecast is for the date 2009.02.16. You are suppsed to solve the tasks using Big Query, which can be used in the Jupyter Notebook like it is shown in the following cell. For further information and how to used BigQuery in Jupyter Notebook refer to the Google Docs. 

The goal of this test is, to test your coding knowledge in Python, BigQuery and Pandas as well as your understanding of Data Science. If you get stuck at the first part, you can use the replacement data provided in the second part

In [ ]:
%%bigquery
SELECT
*,
FROM `bigquery-public-data.samples.gsod`
LIMIT 20 

## Part 1

### 1. Task
Change the date format to 'YYYY-MM-DD' and select the data from 2005 till 2009 for station numbers including and between 725300 and 726300 , and save it as a pandas dataframe. Note the maximum year available is 2010. 

In [ ]:
%%bigquery weather_data_raw
SELECT
    station_number,
    mean_temp,
    num_mean_temp_samples,
    mean_dew_point,
    num_mean_dew_point_samples,
    mean_sealevel_pressure,
    num_mean_sealevel_pressure_samples,
    mean_station_pressure,
    num_mean_station_pressure_samples,
    mean_visibility,
    num_mean_visibility_samples,
    mean_wind_speed,
    num_mean_wind_speed_samples,
    max_sustained_wind_speed,
    max_gust_wind_speed,
    max_temperature,
    max_temperature_explicit,
    total_precipitation,
    snow_depth,
    fog,
    rain,
    snow,
    hail,
    thunder,
    tornado,
    CONCAT(year, '-', month, '-', day) as date
FROM 
    `bigquery-public-data.samples.gsod`
WHERE
    year BETWEEN 2005 AND 2009 AND
    station_number BETWEEN 725300 AND 726300

In [ ]:
# Your instructions read like I am supposed to convert dates to a string in the given format, but I'd rather 
# workwith pandas native datetime format, as it makes selecting ranges easier.
weather_data_raw['date'] = pd.to_datetime(weather_data_raw['date'])
weather_data_raw.head()

### 2. Task 
From here want to work with the data from all stations 725300 to 725330 that have information from 2005 till 2009. 

In [ ]:
weather_data = weather_data_raw.query('725300 <= station_number <= 725330')
weather_data.info()

Do a first analysis of the remaining dataset, clean or drop data depending on how you see appropriate. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

date_index = pd.date_range(weather_data['date'].min(), weather_data['date'].max())
print(f'We have {len(date_index)} days in the dataset.')
for key, group in weather_data.groupby(['station_number']):
    print(f'Temperature data for station {key} has {len(group.mean_temp)} entries, e.g. data for {len(date_index) - len(group.mean_temp)} days is missing.')

print('Lets see where some of those missing dates are.')

def plot_missing_data(column, axis, weather_data):

    dates = pd.DataFrame(columns=[station for station in weather_data['station_number'].unique()], index=date_index, data=0)
    for key, group in weather_data.groupby(['station_number']):
        dates_with_non_null_mean_temp = weather_data[~weather_data[column].isna()][weather_data['station_number']==key]
        dates.loc[dates_with_non_null_mean_temp['date'].values, key] = 1
    dates.plot(ax=axis)
    axis.set_title(f'Days with missing data in {column}')
    
    
fig, axes = plt.subplots(ncols=3, figsize=(18, 5)) 

plot_missing_data('mean_temp', axes[0], weather_data)
plot_missing_data('snow', axes[1], weather_data)
plot_missing_data('total_precipitation', axes[2], weather_data)
weather_data.head()

Random values are distributed across the dataset and not connected periods of time, so an educated guess would be to fill them with sensible defaults.

For demperature and pressure time series etc., where the process underlying the measurements is continuous and differentiable, I interpolate the missing values. For boolean variables such as snow, I fill them with with values from a timestep before. This does not skewing the overal distribution but it does increase autocorrelation however which will influence our forecasting results. If I was to spend more time on this, I would check by how much and experiment with different filling methods here.

In [ ]:
from itertools import product

# Reindex on station_number and date to get continuous time series.
weather_data_reindexed = weather_data.set_index(["station_number", "date"]).reindex(
    pd.MultiIndex.from_product(
        [weather_data["station_number"].unique(), date_index],
        names=["station_number", "date"],
    ),
    fill_value=float('nan')
)

# Interpolate values for variables for which the underlying process is assumed to be smooth.
weather_data_reindexed['mean_temp'].interpolate(inplace=True)
weather_data_reindexed['mean_dew_point'].interpolate(inplace=True)
weather_data_reindexed['mean_sealevel_pressure'].interpolate(limit_directionlimit_direction='both', inplace=True)
weather_data_reindexed['mean_visibility'].interpolate(inplace=True)
weather_data_reindexed['mean_wind_speed'].interpolate(inplace=True)
weather_data_reindexed['max_sustained_wind_speed'].interpolate(inplace=True)
weather_data_reindexed['max_temperature'].interpolate(inplace=True)
weather_data_reindexed['total_precipitation'].interpolate(inplace=True)

# Assume that, if there was snow, they'd report it
weather_data_reindexed['snow_depth'].fillna(value=0, inplace=True)  

# Assume data quality is approximately constant for each station without double checking though.
weather_data_reindexed['num_mean_temp_samples'].fillna(method='ffill', inplace=True)
weather_data_reindexed['num_mean_dew_point_samples'].fillna(method='ffill', inplace=True)
weather_data_reindexed['num_mean_visibility_samples'].fillna(method='ffill', inplace=True)
weather_data_reindexed['num_mean_wind_speed_samples'].fillna(method='ffill', inplace=True)

# Fill with previous value, keeps overall distribution approx. same but increases autocorrelation
weather_data_reindexed['fog'].fillna(method='ffill', inplace=True)
weather_data_reindexed['rain'].fillna(method='ffill', inplace=True)
weather_data_reindexed['snow'].fillna(method='ffill', inplace=True)
weather_data_reindexed['hail'].fillna(method='ffill', inplace=True)
weather_data_reindexed['thunder'].fillna(method='ffill', inplace=True)
weather_data_reindexed['tornado'].fillna(method='ffill', inplace=True)

# Drop columns where there are still nans as data quality is too low.
weather_data_reindexed.dropna(inplace=True, axis=1)

# I suppose, the day of the year could also be a relevant info for the model to guess the likelyhood of snow.
weather_data_reindexed = weather_data_reindexed.reset_index()
weather_data_reindexed['day_of_year'] = weather_data_reindexed['date'].apply(lambda col: pd.Period(col, freq='D').dayofyear)

for column in ['fog', 'rain', 'snow', 'hail', 'thunder', 'tornado']:
    weather_data_reindexed[column] = weather_data_reindexed[column].astype(int)


weather_data_cleaned = weather_data_reindexed
weather_data_cleaned

### 3. Task
Now it is time to split the data, into a training, evaluation and test set. As a reminder, the date we are trying to predict snow fall for is the following, and hence should constitute your test set.

In [ ]:
from datetime import datetime, timedelta

today_12_years_ago = str(datetime.today()- timedelta(days=12*365)).split(' ')[0]
a_week_ago_12_years_ago = str(datetime.today()- timedelta(days=12*365) - timedelta(days=8)).split(' ')[0]

In [ ]:
training_set = weather_data_cleaned[weather_data_cleaned['date']<'2009-09-01']
evaluation_set = weather_data_cleaned[weather_data_cleaned['date']>='2009-09-01']
test_set = weather_data_cleaned[weather_data_cleaned['date']<=today_12_years_ago][weather_data_cleaned['date']>=a_week_ago_12_years_ago]

In [ ]:
from tqdm.notebook import tqdm
import tensorflow as tf

def generate_training_data(training_days: int, dataframe: pd.DataFrame):
    tmp = dataframe.sort_values(['date', 'station_number'])
    tmp_no_dates = tmp.drop(['date', 'station_number'], axis=1)
    date_range = pd.date_range(tmp['date'].min(), tmp['date'].max())
    x_values = []
    y_values = []
    for i in tqdm(range(len(date_range)-(training_days+1))):
        days = date_range[i:i+training_days+1]
        x = tmp_no_dates[dataframe['date'].isin(days[:training_days])].values
        y = tmp_no_dates['snow'][dataframe['date'] == days[-1]].values
        x_values.append([item for sublist in x for item in sublist])
        y_values.append(list(y))
    
    return tf.keras.utils.normalize(x_values, axis=1), tf.keras.utils.normalize(y_values, axis=1)

forecasting_days = 3

x_train, y_train = generate_training_data(forecasting_days,  training_set)
x_eval, y_eval = generate_training_data(forecasting_days, evaluation_set)
x_test, y_test = generate_training_data(forecasting_days, test_set)

## Part 2
If you made it up to here all by yourself, you can use your prepared dataset to train an Algorithm of your choice to forecast whether it will snow on the following date for each station in this dataset:

In [ ]:
import datetime

str(datetime.datetime.today()- datetime.timedelta(days=12*365)).split(' ')[0]

You are allowed to use any library you are comfortable with such as sklearn, tensorflow keras etc. 
If you did not manage to finish part one feel free to use the data provided in 'coding_challenge.csv' Note that this data does not represent a solution to Part 1. 

In [ ]:


modelNN = tf.keras.models.Sequential()

# Flatten inputs for multiple days
modelNN.add(tf.keras.layers.Flatten())

# Input Layers - relu, because it seems to be a good default
modelNN.add(tf.keras.layers.Dense(len(x_train[0]), activation=tf.nn.relu))

# Hidden Layer
modelNN.add(tf.keras.layers.Dense(2*len(y_train[0]), activation=tf.nn.relu))
modelNN.add(tf.keras.layers.Dense(3*len(y_train[0])//2, activation=tf.nn.relu))

# Output Layer - sigmoid for binary classification
modelNN.add(tf.keras.layers.Dense(len(y_train[0]), activation=tf.nn.relu))


modelNN.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4, decay=1e-6),
    loss='mse',
    metrics=["accuracy"],
)

In [ ]:
%tensorboard --logdir logs/fit --port=8080

log_dir = "logs/fit/" + 'NN' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

modelNN.fit(
    x_train,
    y_train,
    validation_data=(x_eval, y_eval),
    epochs=1000,
    verbose=0,
    callbacks=[tensorboard_callback],
)

Converging slowly but looking at the evaluation sample shows that its overfitting.

Try a recurring neural network. Maybe that works better for time series data.

In [ ]:
def generate_training_data_squences(training_days: int, dataframe: pd.DataFrame):
    tmp = dataframe.sort_values(['date', 'station_number'])
    tmp_no_dates = tmp.drop(['date', 'station_number'], axis=1)
    date_range = pd.date_range(tmp['date'].min(), tmp['date'].max())
    x_values = []
    y_values = []
    for i in tqdm(range(len(date_range)-(training_days+1))):
        days = date_range[i:i+training_days+1]
        x = tmp_no_dates[dataframe['date'].isin(days[:training_days])].values
        y = tmp_no_dates['snow'][dataframe['date'] == days[-1]].values
        
        # don't flatten data for several days but keep it as sqeuences for LSTM
        x_values.append(x)
        y_values.append(y)
    
    return tf.keras.utils.normalize(x_values, axis=1), tf.keras.utils.normalize(y_values, axis=1)

forecasting_days = 7

x_train, y_train = generate_training_data_squences(forecasting_days,  training_set)
x_eval, y_eval = generate_training_data_squences(forecasting_days, evaluation_set)
x_test, y_test = generate_training_data_squences(forecasting_days, test_set)

In [ ]:
x_train[0].shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, RepeatVector, TimeDistributed

modelRNN = Sequential()
modelRNN.add(
    LSTM(128, input_shape=(x_train.shape[1:]), activation="relu", return_sequences=True)
)
modelRNN.add(Dropout(0.2))


modelRNN.add(LSTM(50, activation="relu"))

modelRNN.add(Dense(10, activation="sigmoid"))

In [ ]:

modelRNN.compile(
    optimizer='adam',
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

log_dir = "logs/fit/" + 'RNN' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

modelRNN.fit(
    x_train,
    y_train,
    epochs=100,
    validation_data=(x_eval, y_eval),
    callbacks=[tensorboard_callback],
)